# Import

In [1]:
import os
import numpy as np
import random
import time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope

# Build graph

In [2]:
g = tf.get_default_graph()

## Define preprocessing

In [3]:
with tf.name_scope('Preprocessing'):
    input_im = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_im')
    resized_im = tf.image.resize_bilinear(tf.image.convert_image_dtype(
        tf.convert_to_tensor(input_im) ,dtype=tf.float32), [299,299], name='resized_im')
    normalized_im = tf.mul(tf.sub(resized_im, 0.5), 2.0, name='normalized_im')

## Load Inception-v3 graph from slim

In [4]:
with slim.arg_scope(inception_v3_arg_scope()):
    logits, end_points = inception_v3(normalized_im, num_classes=1001, is_training=False)
slim_variables = [v for v in tf.global_variables() if v.name.startswith('InceptionV3/')]

## Create own branch

In [5]:
with tf.name_scope('Own'):
    y = tf.placeholder("float")
    x = tf.reshape(g.get_tensor_by_name('InceptionV3/Logits/SpatialSqueeze:0'),(1,1001))
    w = tf.Variable(tf.random_normal([int(x.get_shape()[-1]),300], stddev=float('1e-5')), name='weights')
    b = tf.Variable(tf.random_normal([1,300]), name='bias')
    y_pred = tf.add(tf.matmul(x, w), b, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)

# Read data

In [6]:
# Images
val_image_path = '../../data/val2014/'
val_image_list = os.listdir(val_image_path)

# Feed through

In [8]:
with tf.Session() as sess:
    
    # Initialize variables
    timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
    saver = tf.train.Saver()
    latest_checkpoint = tf.train.latest_checkpoint(
        "./slim_inception_v3/runs/20170213-1508/checkpoint/")
    saver.restore(sess, latest_checkpoint)
    
    # Single-threaded
    new_vec_dict = {}
    for i, im in enumerate(val_image_list[:1000]):
        image = [np.array(Image.open(val_image_path + im))]
        try:
            new_vec_dict[im] = sess.run(y_pred,feed_dict={input_im:image})
        except ValueError:
            pass
        if i % 10 == 0:
            print(i, end=' ')
    
    save_path = '../../data/image_space_'+timestamp+'.npy'
    np.save(save_path,new_vec_dict)
    print('\n\nImage space saved to {}'.format(save_path))

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 

Image space saved to ../../data/image_space_20170213-1541.npy
